In [1]:
import os
import pandas as pd
import numpy as np
from haversine import haversine

TRAIN_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN/'
TRAIN_AWS_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_AWS/'
META_AWS_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/META/awsmap.csv'
META_PM_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/META/pmmap.csv'

In [2]:
print(len(os.listdir(TRAIN_PATH)))
print(len(os.listdir(TRAIN_AWS_PATH)))

17
30


In [3]:
df_meta_aws = pd.read_csv(META_AWS_PATH)
df_meta_pm = pd.read_csv(META_PM_PATH)

df_meta_aws.rename(columns={'Location':'Location_aws', 'Latitude':'Latitude_aws', 'Longitude':'Longitude_aws', 'Description':'Description_aws'}, inplace= True)
df_meta_pm.rename(columns={'Location':'Location_pm', 'Latitude':'Latitude_pm', 'Longitude':'Longitude_pm', 'Description':'Description_pm'}, inplace= True)

df_meta_aws.shape, df_meta_pm.shape

((30, 4), (17, 4))

In [4]:
merged_df = pd.merge(df_meta_pm,df_meta_aws,how='cross')
merged_df.shape

(510, 8)

In [5]:
merged_df.describe()

,Latitude_pm,Longitude_pm,Latitude_aws,Longitude_aws
count,510.000000,510.000000,510.000000,510.000000
mean,36.575567,126.987315,36.523330,126.813013
std,0.230473,0.367956,0.246121,0.460106
min,36.199217,126.280501,36.062500,125.559500
25%,36.368242,126.655087,36.324400,126.502100
50%,36.592887,127.119209,36.525750,126.862800
75%,36.782355,127.292550,36.690800,127.200500
max,36.987579,127.437825,37.010600,127.493800


In [6]:
merged_df['id'] = merged_df['Location_pm'] + '_' + merged_df['Location_aws']

# 유클리디안
# merged_df['distance'] = ((merged_df['Latitude_pm'] - merged_df['Longitude_pm'])**2 + (merged_df['Latitude_aws'] - merged_df['Longitude_aws'])**2)**(1/2)

# Haversine : haversine( (row, row), (row, row) )
# 속도 비교 글 https://lamttic.github.io/2020/03/11/01.html

merged_df['distance'] = merged_df.apply(lambda row : haversine( (row['Latitude_pm'], row['Longitude_pm']) , (row['Latitude_aws'], row['Longitude_aws'])) , axis=1)


In [7]:
def set_train_dataset(train_df_name,merged_df):

    col_lst = ['temp','wind_direction','wind_speed','rain','humid']

    df = pd.read_csv(TRAIN_PATH+train_df_name+'.csv')
    df.rename(columns={'연도':'Date', '일시':'Time','측정소':'Place'},inplace=True)
    df[col_lst] = 0
    distance_sum = 0



    for aws in os.listdir(TRAIN_AWS_PATH):
        temp_df = pd.read_csv(TRAIN_AWS_PATH+aws)
        temp_df.rename(columns={'기온(°C)':'temp','풍향(deg)':'wind_direction','풍속(m/s)':'wind_speed','강수량(mm)':'rain','습도(%)':'humid'},inplace=True)

        distance = merged_df[(merged_df.Location_aws==aws.split('.')[0]) & (merged_df.Location_pm==train_df_name)].distance.values[0]

        distance_sum += distance
        df[col_lst] = df[col_lst].add(temp_df[col_lst]*distance)
    
    df[col_lst] /= distance_sum

    return df

In [9]:
pm_lst = ['공주','노은동','논산','대천2동','독곶리','동문동','모종동','문창동','성성동','신방동','신흥동','아름동','예산군','읍내동','이원면','정림동','홍성읍']
print(len(pm_lst))

17


In [10]:
df = set_train_dataset('공주',merged_df)

for pm in pm_lst[1:]:
    temp = set_train_dataset(pm,merged_df)
    df = pd.concat([df,temp])

df.shape

(596088, 9)